# Vector space model

In [36]:
import numpy as np
import pandas as pd
import re

cities = pd.read_csv('movies.csv')
cities.head()

,title,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


### Word embeddings

In [37]:
def tokenize(text: str) -> list[str]:
    return re.findall(r"[A-Za-z]+", str(text).lower())

def compute_vocabulary(file: str) -> dict[str, int]:
    vocabulary: dict[str, int] = {}  
    data = pd.read_csv(file)
    i = 0
    for _, row in data.iterrows():
        city = tokenize(row['title'])
        for token in city:
            if token not in vocabulary:
                vocabulary[token] = i
                i += 1
            else: continue
    return vocabulary   

In [38]:
vocab = compute_vocabulary('movies.csv')
print(vocab)
print(len(vocab))

{'creed': 0, 'iii': 1, 'avatar': 2, 'the': 3, 'way': 4, 'of': 5, 'water': 6, 'super': 7, 'mario': 8, 'bros': 9, 'movie': 10, 'mummies': 11, 'supercell': 12, 'cocaine': 13, 'bear': 14, 'john': 15, 'wick': 16, 'chapter': 17, 'puss': 18, 'in': 19, 'boots': 20, 'last': 21, 'wish': 22, 'attack': 23, 'on': 24, 'titan': 25, 'park': 26, 'winnie': 27, 'pooh': 28, 'blood': 29, 'and': 30, 'honey': 31, 'exorcist': 32, 'murder': 33, 'mystery': 34, 'black': 35, 'panther': 36, 'wakanda': 37, 'forever': 38, 'pope': 39, 's': 40, 'prizefighter': 41, 'life': 42, 'jem': 43, 'belcher': 44, 'knock': 45, 'at': 46, 'cabin': 47, 'devil': 48, 'conspiracy': 49, 'cazadora': 50, 'gold': 51, 'run': 52, 'magician': 53, 'elephant': 54, 'plane': 55, 'passion': 56, 'christ': 57, 'batman': 58, 'doom': 59, 'that': 60, 'came': 61, 'to': 62, 'gotham': 63, 'shazam': 64, 'fury': 65, 'gods': 66, 'consecration': 67, 'shark': 68, 'side': 69, 'moon': 70, 'adam': 71, 'money': 72, 'shot': 73, 'pornhub': 74, 'story': 75, 'm': 76, '

## All together

In [39]:
from scipy.sparse import coo_matrix

class InvertedIndex:
    def __init__(self) -> None:
        self.inverted_lists_: dict[str, list[tuple[int, float]]] = {}
        self.td_matrix_ = None
        self.td_sparse_matrix_ = None 
        self.vocab: dict[str, int] = {}

        self.doc_ids_: list[int] = []
        self.movie_titles_: list[str] = []

    def tokenize(self, text: str) -> list[str]:
        return re.findall(r"[A-Za-z]+", str(text).lower())
    
    def build_from_file(self, file: str) -> None:
        df = pd.read_csv(file)
        doc_id: int = 1
        for _, row in df.iterrows():
            title, genre ,overview, crew = self.tokenize(row['title']), self.tokenize(row['genre']), self.tokenize(row['overview']), self.tokenize(row['crew'])
            for word in title + overview + crew + genre:
                if word not in self.inverted_lists_:
                    self.inverted_lists_[word] = [(doc_id, 1)]
                    self.vocab[word] = len(self.inverted_lists_) - 1
                else:
                    if self.inverted_lists_[word][-1][0] != doc_id:
                        self.inverted_lists_[word].append((doc_id, 1))
                    else:
                        new_score = self.inverted_lists_[word][-1][1] + 1
                        self.inverted_lists_[word].append((doc_id, new_score))
            self.doc_ids_.append(doc_id)
            self.movie_titles_.append(row['title'])
            doc_id += 1

        # Time for linear algebra!
        # Build td matrix.
        self.td_matrix_ = np.zeros((len(self.inverted_lists_), len(self.doc_ids_))) # num_unique_words x num_documents
        for row, word in enumerate(self.inverted_lists_):
            for doc_id, tf in self.inverted_lists_[word]:
                self.td_matrix_[row][doc_id-1] = tf

        self.td_sparse_matrix_ = coo_matrix(self.td_matrix_)

    def process_query(self, query: str) -> list[tuple[int, int]]:
        query_words = self.tokenize(query)
        query_vector = np.zeros(len(self.inverted_lists_))
        
        for word in query_words:
            if word in self.inverted_lists_:
                query_vector[self.vocab[word]] += 1
        # Sparse matrix for fast computation. 
        query_scores = query_vector.T @ self.td_sparse_matrix_
        # Slow variant. 
        # query_scores = query_vector.T @ self.td_matrix_
        results = list(sorted([(i+1, score) for i, score in enumerate(query_scores)], key=lambda x: x[1], reverse=True))
        return results

    def generate_output(self, query: str, print_threshold: int = 10) -> None:
        results = self.process_query(query)
        print(f"Query: {query}") 
        for i, (doc_id, score) in enumerate(results):
            print(f"{self.movie_titles_[doc_id-1]}")
            if i > print_threshold:
                break

### Sparse matrix for fast computation

In [40]:
SearchEngine = InvertedIndex()
SearchEngine.build_from_file('movies.csv')
print(SearchEngine.td_matrix_)
print(SearchEngine.td_sparse_matrix_)

  (0, 0)	5.0
  (0, 114)	3.0
  (0, 115)	3.0
  (0, 774)	1.0
  (0, 1030)	1.0
  (0, 1380)	2.0
  (0, 1555)	3.0
  (0, 2083)	1.0
  (0, 2996)	5.0
  (0, 2997)	5.0
  (0, 3073)	1.0
  (0, 3096)	1.0
  (0, 4103)	6.0
  (0, 4104)	6.0
  (0, 5709)	2.0
  (0, 6723)	3.0
  (0, 7478)	1.0
  (0, 8097)	1.0
  (0, 8322)	2.0
  (0, 8428)	2.0
  (1, 0)	1.0
  (1, 26)	1.0
  (1, 165)	2.0
  (1, 277)	1.0
  (1, 430)	1.0
  :	:
  (70422, 10168)	1.0
  (70423, 10169)	1.0
  (70424, 10169)	1.0
  (70425, 10169)	1.0
  (70426, 10170)	2.0
  (70427, 10170)	1.0
  (70428, 10170)	1.0
  (70429, 10171)	1.0
  (70430, 10172)	1.0
  (70431, 10172)	1.0
  (70432, 10172)	1.0
  (70433, 10172)	1.0
  (70434, 10172)	1.0
  (70435, 10172)	1.0
  (70436, 10172)	2.0
  (70437, 10172)	1.0
  (70438, 10174)	1.0
  (70439, 10174)	1.0
  (70440, 10174)	1.0
  (70441, 10175)	1.0
  (70442, 10175)	1.0
  (70443, 10176)	2.0
  (70444, 10176)	1.0
  (70445, 10176)	1.0
  (70446, 10177)	1.0


### Query: Iron Man

In [41]:
SearchEngine.generate_output('iron man')

Query: iron man
Iron Man & Captain America: Heroes United
Iron Man & Hulk: Heroes United
Iron Man 2
Life of Brian
Iron Man: Rise of Technovore
Spider-Man: Across the Spider-Verse
Spider-Man: Homecoming
Iron Man 3
Iron Man
Raped by an Angel
3-Iron
LEGO Marvel Super Heroes: Avengers Reassembled!


### Query: Action

In [42]:
SearchEngine.generate_output('action')

Query: action
Last Action Hero
Bruce Lee: A Warrior's Journey
Missing in Action 2: The Beginning
Shazam! Fury of the Gods
Die Hart
Memory
Avengers: Age of Ultron
Incredibles 2
Rambo
The Action Pack Saves Christmas
Rush Hour
I Am Number Four


### Query: Love

In [43]:
SearchEngine.generate_output('love')

Query: love
Love Me, Love Me Not
Please Enlighten Me
The Moment You Fall in Love
Urusei Yatsura: Always My Darling
Billboard Dad
Sweet Sex and Love
Till We Meet Again
Love Affair
Endless Love
AI Love You
I've Always Liked You
A Frozen Flower


### Query: Anime

In [44]:
SearchEngine.generate_output('Anime')

Query: Anime
Kimetsu Orchestra Concert
The Animatrix
Laid-Back Camp The Movie
ODDTAXI in the Woods
Dragon Ball: Yo! Son Goku and His Friends Return!!
Death Note Relight 1: Visions of a God
Halo Legends
Naruto to Boruto: The Live 2019
Evangelion: 1.0 You Are (Not) Alone
Steins;Gate: The Movie - Load Region of Déjà Vu
Phantom of the Kill: Zero's Rebellion
Death Note Relight 2: L's Successors
